# OpenAI Responses API: Advanced Tweet Analysis with File & Web Search Integration

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Connecting to a MongoDB database** to store and retrieve tweets.
- **Extracting tweets** and converting them into a structured format for further analysis.
- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval 
and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API and interact with a MongoDB database, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API
- **`pymongo`**: A Python driver for MongoDB to store and retrieve tweets.

In [1]:
pip install openai pymongo -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
# tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Tweets from MongoDB

In [5]:
filter={

    
}
project={
    'tweet.text': 1,
    '_id':0
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

After retrieving tweets from MongoDB, we convert the query result into a list format for easier processing.
The data is then serialized into a JSON-formatted string, ensuring it can be properly stored and shared across different services.
Using `io.BytesIO`, we create an in-memory JSON file, eliminating the need for disk writes.
This approach is particularly useful for applications that require temporary file storage, such as uploading datasets
to OpenAI's file search API or cloud storage for further analysis.

In [6]:
result_list = list(result)

# Convert result list to JSON string
json_data = json.dumps(result_list, default=str, indent=4)

# Create an in-memory JSON file
json_bytes = io.BytesIO(json_data.encode("utf-8"))
json_bytes.name = "tweet.json" 

In [7]:
print('Number of tweets: ',len(result_list))

Number of tweets:  98


## Initialize OpenAI Client

In [8]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [9]:
vector_store = client.vector_stores.create(
    name="tweet_base"
)
vector_store_id = vector_store.id
# print(vector_store_id)

### Upload Tweets File

In [10]:
file = client.files.create(
            file=json_bytes,
            purpose="assistants",)

file_id = file.id
# print(file_id)

### Attach File to Vector Store

In [11]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

# print(attach_status.id)

### Query the Vector Store

In [12]:
query = "the latest development in genai"

In [13]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

Gen-4\u2026"
        }
    },
    {
        "tweet": {
            "text": "RT @HorrorHijabi: how it
 Relevant score: 0.6199752008333217
Gen-4\u2026"
        }
    },
    {
        "tweet": {
            "text": "RT @HorrorHijabi: how it
 Relevant score: 0.597909564664309
[
    {
        "tweet": {
            "text": "RT @16pxl: just posting my ghibli inspired pixel art
 Relevant score: 0.5531505036226959
Sans formation pr\u00e9alable,on p\u2026"
        }
    },
    {
        "tweet": {
            "tex
 Relevant score: 0.5457443404105708
}
    },
    {
        "tweet": {
            "text": "Big congrats to @MahojinAI, generative AI rem
 Relevant score: 0.5168808760277529


## OpenAI Response API

### Simple Response

In [14]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [15]:
display(Markdown(simple_response.output_text))

As of my last update, some key developments in Generative AI include:

1. **Advanced Language Models**: AI models like GPT-4 and beyond continue to improve in terms of understanding and generating human-like text, with enhancements in context awareness and creativity.

2. **Multimodal Models**: These models can process and generate different types of data simultaneously, like text, images, and audio, leading to more versatile AI applications.

3. **Real-time Interaction**: Generative AI is increasingly used for dynamic content generation in games and interactive media, allowing for personalized experiences.

4. **Fine-tuning and Customization**: More tools and platforms are available for fine-tuning AI models for specific tasks or industries, enabling broader adoption across different sectors.

5. **Ethical and Responsible AI**: There is ongoing work to address biases and ensure the ethical use of generative AI, with efforts focusing on transparency and accountability.

6. **Creative Applications**: AI-generated art, music, and writing are gaining popularity, sparking discussions on creativity and ownership.

7. **Efficiency Improvements**: New methods are being developed to make generative AI models more computationally efficient, reducing costs and environmental impact.

These advancements are shaping the future of AI, making it more accessible and impactful across various fields.

### File Search Response

In [16]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [17]:
display(Markdown(file_search_response.output_text))


Recent developments in generative AI include:

1. **New AI Models**: RunwayML introduced Gen-4, a series of state-of-the-art AI models for media generation and world consistency.

2. **Industry Collaborations**: NVIDIA and AWS are working together to make generative AI more accessible and impactful.

3. **Market Growth**: Software and internet companies are expected to see a significant increase in revenue from generative AI, with a positive return on investment anticipated by 2025.

4. **Generative Video Effects**: AI is increasingly being used to redefine content creation, acting as a collaborator to enhance creative potential.

5. **Infrastructure Developments**: Inferencing is becoming a key driver for generative AI infrastructure, with advancements in model optimization and architecture.

These advancements highlight the growing impact and integration of generative AI across various sectors.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [18]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(web_search_response.output_text))

Generative AI (GenAI) has experienced significant advancements across various sectors, leading to innovative applications and enhanced capabilities. Here are some of the latest developments:

**Gaming and Real-Time AI Rendering**

Microsoft introduced WHAMM (World and Human Action MaskGIT Model), a generative AI model designed for real-time gaming. Demonstrated through a browser-based version of Quake II, WHAMM generates visual outputs rapidly, supporting real-time user interaction. This model utilizes a MaskGIT-style framework to produce image tokens in parallel, reducing latency and computational demands. Although the demo exhibits some limitations, such as low frame rates and input lag, it represents a significant step toward integrating AI into gaming experiences. ([tomshardware.com](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai))

**Business Applications and AI Integration**

At its Cloud Next conference, Google emphasized the practical business applications of its AI technology. Collaborating with over 500 customers, including Mattel, Google showcased how its BigQuery AI tool analyzed feedback on products like the Barbie Dreamhouse. Additionally, Google unveiled new offerings, such as the TPUv7 AI chip and A2A, a proposed protocol for inter-company AI agent communication, underscoring its commitment to providing tangible value in real-world business environments. ([axios.com](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai))

**Advancements in AI Hardware and Robotics**

Nvidia's GTC 2025 event highlighted several innovations, including the introduction of Blackwell Ultra AI chips and the Groot N1 AI model for robotics. The company also unveiled Blue, a robot developed in collaboration with Google DeepMind and Disney, powered by Nvidia's Newton platform. These developments emphasize Nvidia's focus on expanding AI capabilities in hardware and robotics, aiming to shape future advancements in autonomous technologies. ([tomsguide.com](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai))

**3D Content Generation Tools**

Tencent released a suite of open-source AI tools capable of converting text and images into 3D visuals. Leveraging its Hunyuan3D-2.0 technology, these models can generate high-quality 3D visuals in approximately 30 seconds. This initiative reflects the growing competition among Chinese firms to offer high-performing AI models at reduced costs, particularly targeting designers and game developers. ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai))

**Advancements in Language Models**

Google DeepMind released Gemini, a multimodal large language model and successor to LaMDA and PaLM 2. Gemini comes in three sizes: Nano, Pro, and Ultra, and integrates into Google's chatbot, formerly known as Bard. Subsequent updates, such as Gemini 2.0 Flash and Gemini 2.5, introduced expanded multimodality, including image and audio generation, and enhanced reasoning capabilities, marking significant progress in AI language models. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_DeepMind?utm_source=openai))


## Recent Breakthroughs in Generative AI:
- [You can now play a real-time AI-rendered Quake II in your browser - Microsoft's WHAMM offers generative AI for games](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai)
- [Google touts AI's real-world business model](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai)
- [Nvidia GTC 2025 - Blackwell Ultra, Groot N1, self-driving cars and more from Jensen Huang's keynote](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai) 

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [20]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Generative AI (GenAI) has experienced significant advancements across various sectors, leading to in

### Continue Query with Web Search

In [21]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [22]:
display(Markdown(continue_search_response.output_text))

Generative AI (GenAI) continues to evolve rapidly, influencing various sectors and prompting significant developments. Here are some recent advancements:

**Amazon's Alexa Transformation**

Amazon is reengineering its Alexa digital assistant by integrating generative AI, aiming to enhance its capabilities from simple tasks to functioning as a personalized concierge. This overhaul addresses challenges like AI "hallucinations," response speed, and reliability, with considerations for child safety filters and financial sustainability through potential subscription services. ([ft.com](https://www.ft.com/content/de4c86b8-c744-4051-9255-d34259223160?utm_source=openai))

**TIME's AI Integration in Journalism**

TIME magazine has launched TIME AI, a platform developed with Scale AI, to revolutionize journalism engagement. Utilizing generative AI, this initiative offers personalized and interactive storytelling experiences, including customized content formats and multilingual support, reflecting a significant advancement in merging journalism with cutting-edge technology. ([time.com](https://time.com/7201556/generative-ai-time-journalism/?utm_source=openai))

**Regulatory Developments in AI**

In October 2023, President Biden issued Executive Order 14110, focusing on establishing standards for AI safety, protecting privacy, and promoting innovation. This order directs federal agencies to monitor and mitigate AI risks, develop guidelines, and implement AI technologies safely. Additionally, in November 2023, the U.S., U.K., and 16 other countries entered into a landmark international agreement emphasizing a "secure by design" approach to AI systems development, highlighting the importance of international collaboration in guiding AI development. ([perspectives.babstcalland.com](https://perspectives.babstcalland.com/recent-developments-in-artificial-intelligence/?utm_source=openai))


## Recent Developments in Generative AI:
- [Amazon races to transplant Alexa's 'brain' with generative AI](https://www.ft.com/content/de4c86b8-c744-4051-9255-d34259223160?utm_source=openai)
- [Why We're Introducing Generative AI to TIME's Journalism](https://time.com/7201556/generative-ai-time-journalism/?utm_source=openai)
- [🏃‍➡️ Race for AI](https://www.axios.com/newsletters/axios-ai-plus-5068b580-ac2d-11ef-83c8-b75248cc21d8?utm_source=openai) 

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [23]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [24]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several key advancements and collaborations:

1. **NVIDIA and AWS Collaboration**: NVIDIA and AWS are working together to make generative AI more accessible and impactful. This partnership aims to help organizations leverage AI technologies more effectively.

2. **Runway's Gen-4 Models**: Runway has introduced Gen-4, a new series of state-of-the-art AI models designed for media generation and maintaining world consistency.

3. **Generative AI in Marketing**: Generative AI is being explored for more than just content creation. It is expected to play a significant role in the next generation of marketing use cases.

4. **AI in Healthcare**: There is a growing interest in using generative AI for breakthroughs in AI-powered healthcare, indicating a trend towards more sophisticated applications in the medical field.

5. **Infrastructure and Accessibility**: Companies like Equinix are offering services like the Instant AI Factory, which provides AI-ready data centers globally to accelerate real-time AI agent responses and generative AI workloads.

These developments highlight the expanding role of generative AI across various industries, from media and marketing to healthcare and infrastructure.